In [1]:
import numpy as np
from scipy.stats import poisson

In [2]:
class PDF:
    def __init__(self, values, param = "mu"):
        self.n = len(values)
        self.contents = values
        self.params = [param]
    def __call__(self, x, **parameters):
        assert x < self.n, "out of range"
        for p_name, p_value in parameters.iteritems():
            if p_name in self.params:
                return self.contents[x] * p_value
            else:
                return self.contents[x]

In [3]:
class AddPDF:
    def __init__(self, pdf1, pdf2):
        assert pdf1.n == pdf2.n, "PDFs must have equal number of bins"
        self.n = pdf1.n
        self.pdfs = [pdf1, pdf2]
        self.params = pdf1.params + pdf2.params
    def __call__(self, x, **parameters):
        assert x < self.n
        val = 0
        for pdf in self.pdfs:
            val += pdf(x, **parameters)
        return val
        

In [4]:
sig = PDF([1,2], "mu")
bkg = PDF([1,1], "B")

In [5]:
for i in range(sig.n):
    print sig(i,mu=1.5)

1.5
3.0


In [6]:
model = AddPDF(sig, bkg)

In [7]:
for i in range(sig.n):
    print model(i,mu=.5)

print model.params

1.5
2.0
['mu', 'B']


In [8]:
class llh:
    def __init__(self, meas, pdf):
        self.model = pdf
        self.data = meas
    def __call__(self, **kwargs):
        pred = np.asarray([self.model(i,**kwargs) for i in range(self.model.n)])
        return -poisson.logpmf(self.data, pred).sum()
    

In [9]:
meas = [2,3]
l = llh(meas,model)
print l(mu=1.5,B=1)

2.99344210268


In [10]:
from iminuit import Minuit, describe, Struct

In [11]:
describe(l)

[]

In [12]:
from iminuit.util import make_func_code
x = [1,2,3,4,5]
y = [2,4,6,8,10]# y=2x
class Chi2Functor:
    def __init__(self,f,x,y):
        self.f = f
        self.x = x
        self.y = y
        f_sig = describe(f)
        #this is how you fake function 
        #signature dynamically
        self.func_code = make_func_code(f_sig[1:])#docking off independent variable
        self.func_defaults = None #this keeps np.vectorize happy
    def __call__(self,*arg):
        #notice that it accept variable length
        #positional arguments
        chi2 = sum((y-self.f(x,*arg))**2 for x,y in zip(self.x,self.y))
        return chi2

In [13]:
sig = [1,2]
bkg = [10,10]
def simplePDF(x,mu,B):
    return sig[x]*mu + bkg[x]*B
def linear(x,m,c):
    return m*x+c

In [14]:
print describe(linear)
print describe(simplePDF)

['x', 'm', 'c']
['x', 'mu', 'B']


In [15]:
data = [20.5,21]
pdf_chi2 = Chi2Functor(simplePDF,[0,1],data)

In [16]:
describe(pdf_chi2)

['mu', 'B']

In [17]:
m = Minuit(pdf_chi2)
m.migrad();
print(m.values)

/Users/vince/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: InitialParamWarning: errordef is not given. Default to 1.
  """Entry point for launching an IPython kernel.
/Users/vince/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: InitialParamWarning: Parameter mu does not have initial value. Assume 0.
  """Entry point for launching an IPython kernel.
/Users/vince/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: InitialParamWarning: Parameter mu is floating but does not have initial step size. Assume 1.
  """Entry point for launching an IPython kernel.
/Users/vince/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: InitialParamWarning: Parameter B does not have initial value. Assume 0.
  """Entry point for launching an IPython kernel.
/Users/vince/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: InitialParamWarning: Parameter B is floating but does not have initial step size. Assume 1.
  """Entry point for launch

FCN = 2.57694730663e-22 
 TOTAL NCALL = 33 
 NCALLS = 33 
 
 
 EDM = 2.57732501392e-22 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,mu,0.5,1.41421,0,0,,,
2,B,2,0.223607,0,0,,,


{'mu': 0.4999999999964606, 'B': 1.9999999999994094}


In [18]:
import math
from __future__ import division

In [36]:
class LLHFunctor:
    def __init__(self,f,x,y):
        self.f = f
        self.x = x
        self.y = y
        f_sig = describe(f)
        #this is how you fake function 
        #signature dynamically
        self.func_code = make_func_code(f_sig[1:])#docking off independent variable
        self.func_defaults = None #this keeps np.vectorize happy
    def __call__(self,*arg):
        #notice that it accept variable length
        #positional arguments
        pred = np.asarray([self.f(i,*arg) for i in self.x])
        llh = -sum([self.y[bin_i]*np.log(pred[bin_i]) for bin_i in range(len(self.y))]) + pred.sum()
        return llh

In [37]:
data = [20.5,21]
pdf_llh = LLHFunctor(simplePDF,[0,1],data)
describe(pdf_llh)

['mu', 'B']

In [38]:
fit_args = dict(mu=1.,limit_mu=(0,10),B=1,limit_B=(0,100))
m = Minuit(pdf_llh, **fit_args)
m.migrad();
print(m.values)

/Users/vince/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:2: InitialParamWarning: errordef is not given. Default to 1.
  
/Users/vince/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:2: InitialParamWarning: Parameter mu is floating but does not have initial step size. Assume 1.
  
/Users/vince/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:2: InitialParamWarning: Parameter B is floating but does not have initial step size. Assume 1.
  


FCN = -84.35367006 
 TOTAL NCALL = 69 
 NCALLS = 69 
 
 
 EDM = 1.11491618812e-05 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,mu,0.515419,9.46423,0,0,0.0,10.0,
2,B,1.99902,1.42288,0,0,0.0,100.0,


{'mu': 0.5154192505272659, 'B': 1.9990197355814698}


In [27]:
vals = np.asarray([-1,2])
vals = (vals > 0).all()
print vals

False
